In [1]:
!pip install tensorflow_datasets

     ---------------------------------------- 5.4/5.4 MB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 101.5/101.5 kB 6.1 MB/s eta 0:00:00
     -------------------------------------- 126.4/126.4 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB ? eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 48.1 MB/s eta 0:00:00
     ------------------------------------- 223.6/223.6 kB 14.2 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=1f70ec2f6a74507499ba15037d4bc5a3fe020ae91d349753ac440c07ce5db4b8
  Stored in directory: c:\users\samsung\appdata\local\pip\cache\wheels\68\ee\8d\57af0d8b0d34c2e918ff29d3af02b348db6499bb107caa007e
Successfully built promise
  Attempting uninstall: requests
    Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.19.4 requires tokenizers!=0.11.3,<0.13,>=0.11.1, but you have tokenizers 0.10.3 which is incompatible.
torchvision 0.12.0 requires torch==1.11.0, but you have torch 1.12.1 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
responses 0.18.0 requires urllib3>=1.25.10, but you have urllib3 1.22 which is incompatible.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.27 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.
anaconda-client 1.11.0 requires urllib3>=1.26.4, but you have urllib3 1.22 which is incompatible.


In [3]:
!pip install tensorflow_hub

     ------------------------------------ 100.6/100.6 kB 828.6 kB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow_hub as hub

import numpy as np

In [5]:
# data 불러오기

dataset, info = tfds.load('cassava', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\SAMSUNG\tensorflow_datasets\cassava\0.1.0.incompleteMM54PA\cassava-test.tfrecord*...:   0%|…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\SAMSUNG\tensorflow_datasets\cassava\0.1.0.incompleteMM54PA\cassava-validation.tfrecord*...:…

Dataset cassava downloaded and prepared to C:\Users\SAMSUNG\tensorflow_datasets\cassava\0.1.0. Subsequent calls will reuse this data.


In [9]:
def preprocess_fn(data):
  image = data['image']

  # Normalize [0, 255] to [0, 1]
  image = tf.cast(image, tf.float32)
  image = image / 255.

  # Resize the images to 180 x 180
  image = tf.image.resize(image, (180, 180))

  data['image'] = image
  return data

In [10]:
# train 5656개, test 1885개, validation 1889개

cassava_train = dataset['train']
cassava_test = dataset['test']
cassava_validation = dataset['validation']

In [11]:
# Extend the cassava dataset classes with 'unknown'
class_names = info.features['label'].names + ['unknown']

# Map the class names to human readable names
name_map = dict(
    cmd='Mosaic Disease',
    cbb='Bacterial Blight',
    cgm='Green Mite',
    cbsd='Brown Streak Disease',
    healthy='Healthy',
    unknown='Unknown')

print(len(class_names), 'classes:')
print(class_names)
print([name_map[name] for name in class_names])

6 classes:
['cbb', 'cbsd', 'cgm', 'cmd', 'healthy', 'unknown']
['Bacterial Blight', 'Brown Streak Disease', 'Green Mite', 'Mosaic Disease', 'Healthy', 'Unknown']


In [12]:
cassava_train = dataset['train'].map(preprocess_fn).batch(5656).as_numpy_iterator()
cassava_train = next(cassava_train)
X_train = cassava_train.get('image').copy()
y_train = cassava_train.get('label').copy()

In [13]:
cassava_test = dataset['test'].map(preprocess_fn).batch(1885).as_numpy_iterator()
cassava_test = next(cassava_test)
X_test = cassava_test.get('image').copy()
y_test = cassava_test.get('label').copy()

In [14]:
cassava_validation = dataset['validation'].map(preprocess_fn).batch(1889).as_numpy_iterator()
cassava_validation = next(cassava_validation)
X_validation = cassava_validation.get('image').copy()
y_validation = cassava_validation.get('label').copy()

In [16]:
print(X_train.shape)
print(X_test.shape)
print(X_validation.shape)

(5656, 180, 180, 3)
(1885, 180, 180, 3)
(1889, 180, 180, 3)


In [18]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [19]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 41, 41, 128)       7385

In [23]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5,
)]

In [24]:
model.compile(optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_validation, y_validation),callbacks=callbacks_list)

Epoch 1/20
89/89 [==============================] - 77s 858ms/step - loss: 0.1031 - accuracy: 0.9742 - val_loss: 3.3042 - val_accuracy: 0.5416
Epoch 2/20
89/89 [==============================] - 74s 834ms/step - loss: 0.0314 - accuracy: 0.9959 - val_loss: 3.5835 - val_accuracy: 0.5442
Epoch 3/20
89/89 [==============================] - 72s 815ms/step - loss: 0.0368 - accuracy: 0.9958 - val_loss: 3.5249 - val_accuracy: 0.5421
Epoch 4/20
89/89 [==============================] - 74s 831ms/step - loss: 0.0317 - accuracy: 0.9952 - val_loss: 3.6805 - val_accuracy: 0.5490
Epoch 5/20
89/89 [==============================] - 74s 828ms/step - loss: 0.0427 - accuracy: 0.9956 - val_loss: 3.3714 - val_accuracy: 0.5326
Epoch 6/20
89/89 [==============================] - 73s 819ms/step - loss: 0.0292 - accuracy: 0.9961 - val_loss: 3.3608 - val_accuracy: 0.5648
Epoch 7/20
89/89 [==============================] - 74s 835ms/step - loss: 0.0301 - accuracy: 0.9961 - val_loss: 3.3410 - val_accuracy: 0.5442

In [26]:
pred_labels = model.predict(X_test)
pred_labels = np.argmax(pred_labels, axis=1).tolist()
print(pred_labels)

59/59 [==============================] - 6s 98ms/step
[4, 3, 0, 1, 3, 3, 3, 4, 3, 1, 2, 2, 3, 0, 3, 3, 3, 3, 3, 2, 1, 1, 2, 1, 0, 1, 1, 1, 3, 1, 3, 3, 1, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 2, 3, 3, 1, 3, 1, 3, 2, 1, 4, 0, 3, 2, 0, 3, 3, 1, 3, 0, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 3, 2, 1, 1, 1, 2, 1, 3, 0, 3, 0, 0, 0, 3, 2, 1, 0, 1, 3, 2, 2, 3, 3, 1, 3, 3, 1, 3, 3, 3, 3, 1, 0, 3, 3, 0, 1, 3, 2, 3, 3, 2, 1, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 0, 1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 0, 1, 1, 3, 3, 3, 0, 1, 1, 3, 1, 3, 3, 3, 2, 1, 3, 3, 3, 2, 2, 3, 3, 3, 3, 1, 3, 0, 1, 3, 1, 3, 3, 3, 0, 3, 3, 3, 1, 3, 1, 1, 1, 3, 3, 1, 3, 0, 3, 3, 3, 3, 1, 3, 1, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 0, 1, 0, 3, 2, 3, 0, 3, 3, 3, 1, 0, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 1, 3, 1, 3, 1, 4, 1, 1, 3, 3, 1, 3, 3, 2, 3, 3, 3, 3, 3, 0, 3, 1, 3, 3, 3, 0, 1, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 1, 3, 1, 3, 1, 1, 3, 3, 3, 2, 0, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 2, 2, 

In [27]:
model.evaluate(X_test,y_test)

59/59 [==============================] - 6s 95ms/step - loss: 3.2325 - accuracy: 0.5342


[3.232473134994507, 0.5342174768447876]